In [1]:
# prerequisites:
# pip install polyglot
# pip3 install pyicu
# pip3 install pycld2
# pip3 install morfessor

# https://www.lyonwj.com/blog/entity-extraction-russian-troll-tweets-neo4j
# https://medium.com/hackernoon/install-polyglot-on-mac-3c90445abc1f
# https://stackoverflow.com/questions/64886067/polyglot-importerror-cannot-import-name-locale-from-icu
# http://brandonrose.org/ner2sna

from polyglot.text import Text
import pandas as pd
from tqdm.notebook import tqdm
import json
import sys

from polyglot.detect.base import logger as polyglot_logger
polyglot_logger.setLevel("ERROR")

In [2]:
# test
blob = Text('@realDonaldTrump "Hillary Clinton has zero record to run on - unless you call corruption positive.." - @IngrahamAngle')
blob.entities

[I-PER(['Hillary', 'Clinton'])]

In [3]:
# load and preview tweets data
tweets = pd.read_csv("russian_twitter_trolls/tweets.csv")
tweets.head(2)

user_id        user_key    created_at          created_str  \
0  2.532612e+09       kathiemrr  1.488207e+12  2017-02-27 14:54:00   
1  2.531160e+09  traceyhappymom  1.471273e+12  2016-08-15 14:50:20   

   retweet_count retweeted  favorite_count  \
0            NaN       NaN             NaN   
1            NaN       NaN             NaN   

                                                text      tweet_id source  \
0    #ThingsDoneByMistake kissing auntie in the lips  8.362279e+17    NaN   
1  RT @mc_derpin: #TheOlderWeGet the more pessimi...  7.651989e+17    NaN   

                  hashtags expanded_urls  posted mentions  \
0  ["ThingsDoneByMistake"]            []  POSTED       []   
1        ["TheOlderWeGet"]            []  POSTED       []   

   retweeted_status_id  in_reply_to_status_id  
0                  NaN                    NaN  
1                  NaN                    NaN

In [4]:
# load and preview users data
users = pd.read_csv("russian_twitter_trolls/users.csv")
users.head(2)

id                       location              name  \
0  100345056.0  still ⬆️Block⤵️Corner⬇️street  #Ezekiel2517✨...   
1  247165706.0                    Chicago, IL  B E C K S T E R✨   

   followers_count  statuses_count                    time_zone verified lang  \
0           1053.0         31858.0                          NaN    False   en   
1            650.0          6742.0  Mountain Time (US & Canada)    False   en   

   screen_name                                        description  \
0  SCOTTGOHARD  CELEBRITY TRAINER ✨#424W147th✨ #CrossfitCoach ...   
1  Beckster319  Rebecca Lynn Hirschfeld Actress.Model.Writer.A...   

                       created_at  favourites_count  friends_count  \
0  Tue Dec 29 23:15:22 +0000 2009            2774.0         1055.0   
1  Fri Feb 04 06:38:45 +0000 2011            7273.0          896.0   

   listed_count  
0          35.0  
1          30.0

In [5]:
tweets2 = tweets.loc[~tweets.text.isnull()]  # drop 20 rows with missings in text column

In [6]:
entityArr = []

err_num = 0

for user_id, tweet_id, text in tqdm(zip(tweets2.user_id, tweets2.tweet_id, tweets2.text), total=tweets2.shape[0]):    
    try:
        parsedTweet = {}
        parsedTweet['user_id'] = user_id
        parsedTweet['tweet_id'] = tweet_id
        parsedTweet['text'] = text
        blob = Text(text)
        entities = blob.entities
        parsedTweet['entities'] = []
        for ent in entities:
            eobj = {}
            eobj['tag'] = ent.tag
            eobj['entity'] = ent
            parsedTweet['entities'].append(eobj)
        if len(parsedTweet['entities']) > 0:
            entityArr.append(parsedTweet)
    except Exception as exc:
        err_num += 1
        # e_type, e_value, e_traceback = sys.exc_info()
        # for item in e_value.args:
        #     if item.startswith("This resource is available in the index but not downloaded"):
        #         !{item.split("\n\n")[1]}

  0%|          | 0/203430 [00:00<?, ?it/s]

In [7]:
entityArr[:3]

[{'user_id': 2532611755.0,
  'tweet_id': 8.362278918976512e+17,
  'text': '#ThingsDoneByMistake kissing auntie in the lips',
  'entities': [{'tag': 'I-PER', 'entity': I-PER(['auntie'])}]},
 {'user_id': 1694026190.0,
  'tweet_id': 7.775946478750597e+17,
  'text': 'RT @NahBabyNah: Twitchy: Chuck Todd caught out there shilling for Hillary Clinton\r\nThe post BUSTED: Adam Baldwi... https://t.co/ay28pMpDw6 #…',
  'entities': [{'tag': 'I-PER', 'entity': I-PER(['Chuck', 'Todd'])},
   {'tag': 'I-PER', 'entity': I-PER(['Hillary', 'Clinton'])},
   {'tag': 'I-PER', 'entity': I-PER(['Adam', 'Baldwi'])}]},
 {'user_id': 2587100717.0,
  'tweet_id': 7.180400616490312e+17,
  'text': 'RT @mcicero10: #BernieSanders #Trump people should rally TOGETHER against the establishment who is 💩-ing on both choices #thefix',
  'entities': [{'tag': 'I-PER', 'entity': I-PER(['Trump'])}]}]

In [8]:
with open('parsed_tweets.json', 'w') as f:
    json.dump(entityArr, f, ensure_ascii=False, sort_keys=True, indent=4)

In [12]:
20 + err_num  # loss cnt

881

In [13]:
(20 + err_num) / tweets2.shape[0]  # loss ratio

0.00433072801455046